# Simple BYOK Demo

> A minimal example showing basic BYOK functionality

In [1]:
#| default_exp examples.simple_demo

## Basic Setup

In [2]:
from fasthtml.common import *
from cjm_fasthtml_byok.core.storage import BYOKManager
from cjm_fasthtml_byok.middleware.beforeware import create_byok_beforeware
from cjm_fasthtml_byok.components.forms import KeyInputForm
from cjm_fasthtml_byok.components.alerts import Alert

# daisyUI and Tailwind imports
from cjm_fasthtml_daisyui.core.resources import get_daisyui_headers
from cjm_fasthtml_tailwind.utilities.spacing import p as padding
from cjm_fasthtml_tailwind.core.base import combine_classes

## Create Simple App

In [3]:
# Initialize BYOK (session-only mode)
SECRET_KEY = "simple-demo-secret-key"
byok = BYOKManager(secret_key=SECRET_KEY)

# Create beforeware
byok_beforeware = create_byok_beforeware(byok)

# Create app with session support and beforeware
from starlette.middleware.sessions import SessionMiddleware

app, rt = fast_app(
    hdrs=get_daisyui_headers(),
    secret_key=SECRET_KEY,
    sess_cls=SessionMiddleware,  # Enable sessions
    before=byok_beforeware  # Add beforeware back
)

## Define Routes

In [4]:
@rt("/")
def index(req, sess):
    """Home page with key input form"""
    # Check if we have a key stored
    has_openai = byok.has_key(req, "openai")
    
    return Titled(
        "Simple BYOK Demo",
        Div(
            H1("API Key Management Demo", cls="text-2xl font-bold mb-4"),
            
            # Show status
            Alert(
                f"OpenAI key is {'configured ✓' if has_openai else 'not configured'}",
                kind="success" if has_openai else "warning"
            ),
            
            # Key input form
            Div(
                H2("Add/Update OpenAI Key", cls="text-xl font-semibold mt-6 mb-4"),
                KeyInputForm(provider="openai", action="/save-key"),
                cls="max-w-md"
            ),
            
            # Action buttons if key exists
            Div(
                Div(
                    A("Test API Key", href="/test", cls="btn btn-primary mr-2"),
                    A("Visit Protected Route", href="/protected", cls="btn btn-secondary mr-2"),
                    cls="flex gap-2 mb-4"
                ),
                # Delete button (as a form to use POST)
                Form(
                    Button("Delete Key", type="submit", cls="btn btn-error btn-sm"),
                    method="post",
                    action="/delete-key",
                    onsubmit="return confirm('Are you sure you want to delete the key?');"
                )
            ) if has_openai else None,
            
            # Try protected route without key button
            Div(
                P("Try visiting the protected route without a key:", cls="mt-4"),
                A("Visit Protected Route", href="/protected", cls="btn btn-warning")
            ) if not has_openai else None,
            
            cls=combine_classes("container", "mx-auto", padding._6)
        )
    )

In [5]:
@rt("/save-key", methods=["POST"])
def save_key(req, sess, api_key: str):
    """Save the API key"""
    try:
        byok.set_key(req, "openai", api_key)
        return RedirectResponse(url="/?saved=true", status_code=303)
    except Exception as e:
        return RedirectResponse(url="/?error=true", status_code=303)

@rt("/test")
def test_key(req, sess):
    """Test the stored API key"""
    api_key = byok.get_key(req, "openai")
    
    if not api_key:
        return Titled(
            "No Key Found",
            Alert("No OpenAI key found. Please add one first.", kind="error"),
            A("Go Back", href="/", cls="btn btn-primary mt-4")
        )
    
    from cjm_fasthtml_byok.core.security import mask_key
    
    return Titled(
        "API Key Test",
        Div(
            H1("API Key Retrieved Successfully!", cls="text-2xl font-bold mb-4"),
            Alert(
                f"Key found: {mask_key(api_key)}",
                kind="success"
            ),
            P(
                "In a real application, you would now use this key to make API calls.",
                cls="mt-4"
            ),
            A("Go Back", href="/", cls="btn btn-primary mt-4"),
            cls=combine_classes("container", "mx-auto", padding._6)
        )
    )

In [6]:
@rt("/delete-key", methods=["POST"])
def delete_key(req, sess):
    """Delete the stored API key"""
    byok.delete_key(req, "openai")
    return RedirectResponse(url="/?deleted=true", status_code=303)

@rt("/clear-all", methods=["POST"])
def clear_all_keys(req, sess):
    """Clear all stored API keys"""
    byok.clear_keys(req)
    return RedirectResponse(url="/?cleared=true", status_code=303)

In [7]:
# Example of protected route using decorator
from cjm_fasthtml_byok.middleware.beforeware import require_api_key

@rt("/protected")
@require_api_key("openai")
def protected_route(req):
    """Example of a protected route that requires an API key"""
    # If we get here, the key exists
    # Access byok from request scope (added by beforeware)
    byok_manager = req.scope.get('byok', byok)  # Fallback to global
    api_key = byok_manager.get_key(req, "openai")
    
    from cjm_fasthtml_byok.core.security import mask_key
    
    return Titled(
        "Protected Route",
        Div(
            H1("🔒 Protected Content", cls="text-2xl font-bold mb-4"),
            Alert(
                f"You have access! Using key: {mask_key(api_key)}",
                kind="success"
            ),
            P("This route is protected by the @require_api_key decorator.", cls="mt-4"),
            P("If you didn't have an OpenAI key, you'd see an error page.", cls="mt-2"),
            A("Go Back", href="/", cls="btn btn-primary mt-4"),
            cls=combine_classes("container", "mx-auto", padding._6)
        )
    )

## Run the Demo

In [8]:
#| eval: false
from fasthtml.jupyter import *
from cjm_fasthtml_daisyui.core.testing import start_test_server

# Start server
server = start_test_server(app, port=5002)

# Display in notebook
display(HTMX(port=server.port))

print("🚀 Simple BYOK Demo Running!")
print(f"📍 Visit http://localhost:{server.port}")
print("\nTry adding a test key like 'sk-test123'")

🚀 Simple BYOK Demo Running!
📍 Visit http://localhost:5002

Try adding a test key like 'sk-test123'


In [9]:
#| eval: false
# Stop the server
server.stop()